In [2]:
class User:
    def __init__(self, name, label="none"): 
        self.id = name
        self.label = label
        self.advisors = [] #neighbors
        self.providers = []
        self.interactions = {}
        self.total_interactions = 0
        self.provider_raputation = {}
        self.advisor_reputation = {}
        self.decay = 1 #longitivity
        self.phi = 0.5 #w
        self.epsilon = 1 #uncertainity
        self.limit = 0 #tolarance limit
#         self.alpha = 0.8 #consumer confidence
        self.satisfaction = {}
        self.disatisfaction = {}
        self.b_satisfaction = {}
        self.b_disatisfaction = {}
            
    def interact(self, provider, rating):
        if provider.id != self.id:
            if provider in self.providers:
                ratelist = self.interactions[provider]
            else:
                ratelist = []
                self.providers.append(provider)

            ratelist.append(rating)
            self.interactions[provider] = ratelist
            self.total_interactions += 1

            self.eval_provider(provider, rating)
            self.eval_provider(provider, rating, bayes = True)
        else:
            print('I cant interact with myself')
        
        
    def eval_provider(self, provider, rating, bayes = False):
        exp_val_n = len(self.interactions[provider])

        rj = (1- rating) * self.phi #disatisfaction self.phi = w, rj = ds
        sj = (1+ rating) * self.phi #satisfaction self.phi = w  sj = sj
        
        if bayes:
            if provider in self.b_satisfaction:
                cumilative_satisfaction = self.b_satisfaction[provider]
                cumilative_disatisfaction = self.b_disatisfaction[provider]
                cumilative_satisfaction += sj
                cumilative_disatisfaction += rj
            else:
                cumilative_satisfaction = sj
                cumilative_disatisfaction = rj
            self.b_satisfaction[provider] = cumilative_satisfaction
            self.b_disatisfaction[provider] = cumilative_disatisfaction
        else:
            decay_factor = self.decay **(exp_val_n - self.limit)
            rj *= decay_factor
            sj *= decay_factor
            self.limit += 1
            if provider in self.satisfaction:
                cumilative_satisfaction = self.satisfaction[provider]
                cumilative_disatisfaction = self.disatisfaction[provider]
                cumilative_satisfaction += sj
                cumilative_disatisfaction +=rj
            else:
                cumilative_satisfaction = sj
                cumilative_disatisfaction = rj
            self.satisfaction[provider] = cumilative_satisfaction
            self.disatisfaction[provider] = cumilative_disatisfaction
        
        self.get_belief(provider)
    
    def get_belief(self, provider):
            cum_sat = self.satisfaction[provider]
            cum_dis = self.disatisfaction[provider]
            total_sat_dit = cum_dis+cum_sat
            den = total_sat_dit + self.epsilon 
            belief = cum_sat /(den)
            disbelief = cum_dis /(den)
            uncertainity = self.epsilon / (den)
            
        
            return belief, disbelief, uncertainity
        
        
    def provider_raputation(self, provider):
        return provider_raputation[provider]
    
    def get_providers(self):
        return self.providers
        

    def get_similarity(self, advisor):

        
        common_providers = [x for x in advisor.get_providers() if x in self.get_providers()]
        similarity = -100 #note this is just for checking if the advisor has a common vendor
        
        if len(common_providers) > 0:
            cum_dist = 0
            for p in common_providers:
                    
                    a_bel, a_disbel, a_unc = self.get_belief(p)
                    b_bel, b_disbel, b_unc = advisor.get_belief(p)
                    nonsqrt= self.get_distance(advisor, p)
                    dist = nonsqrt** (1/2)
                    dist = dist/(2**(1/2))
                    cum_dist += dist
            meanormilsed_distance = cum_dist/len(common_providers)
            similarity = 1 - meanormilsed_distance
        return similarity
    
    def get_distance(self, advisor, p):
        a_bel, a_disbel, a_unc = self.get_belief(p)
        b_bel, b_disbel, b_unc = advisor.get_belief(p)
        nonsqrt=((a_bel - b_bel)**2) + ((a_disbel - b_disbel)**2) + ((a_unc - b_unc)**2)
        return nonsqrt
            
    def get_global_crediblity(self):
        cum_bel = 0;
        for p in self.providers:
            b ,_ , _ = p.get_belief(self)
            cum_bel += b
        return cum_bel/len(self.providers)
            
    
    def set_adivsors(self, neigbors):
        self.advisors = neigbors
    def add_an_adivsor(self, neigbor):
        self.advisors.append(neigbor)
    
    def get_id(self):
        self.id
    

    def my_bayes(self, p):
        sat = 0
        dis = 0
        if p in self.b_satisfaction:
            sat = self.b_satisfaction[p]
            dis = self.b_disatisfaction[p]
        return sat, dis
    

    
    def p_g_reputation(self, provider, all_advisors, alpha = 0.8, filter_threshold = 0.5):
        sim_count = 0
        gc_count = 0
        p_rep = 0
        g_rep = 0
        for a in all_advisors:
            if a.id != self.id:
                ap_bel, _, _ = a.get_belief(provider)
                sim = self.get_similarity(a)
                if sim != -100: #meaning have no common vendos
                    sim_count += 1
                    if sim > filter_threshold:
                        p_rep += (sim * ap_bel)
                else:
                    gc = a.get_global_crediblity()
                    g_rep += (gc * ap_bel)
        reputation = 0
        
        if sim_count > 0:
            reputation = (alpha * p_rep)/sim_count

        if (len(all_advisors) - sim_count) > 0:
            reputation += ((1 - alpha) * g_rep)/(len(all_advisors) - sim_count)
        return reputation
        
    
        

In [3]:
class Consumer(User):
    def __init__(self, name, label="C"):
        User.__init__(self,name, label)
        

class Producer(User):
    def __init__(self, name, label="P"):
        User.__init__(self,name, label)

# class Advisor(Users):
#     def __init__(self, name):
#         Users.__init__(self,name)

In [4]:
l_consumers = {}
l_producers = {}
consumer_id_start_range = 100
producer_id_start_range = 200
for x in range(11):
    print(consumer_id_start_range+x)
    name ='C'+str(x+1)
    c = Consumer(consumer_id_start_range+x, name)
    l_consumers[consumer_id_start_range+x] = c
    if x % 2 == 0:
        print(producer_id_start_range+x)
        p = Producer(producer_id_start_range+x)
        l_producers[producer_id_start_range+x] = p
    

100
200
101
102
202
103
104
204
105
106
206
107
108
208
109
110
210


In [5]:
len(l_consumers)

11

In [6]:
def agree(c, p, n_interactions, rate=1):
    for i in range(n_interactions):
        c.interact(p, rate)
        p.interact(c, rate)

def disagree(c, p, n_interactions,rate=-1):
    agree(c, p, n_interactions,rate)

def c_agree_p_disagree(c, p, n_interactions, p_rate=1, n_rate=-1):
    for i in range(n_interactions):
        c.interact(p, p_rate)
        p.interact(c, n_rate)

def p_agree_c_disagree(c, p, n_interactions, p_rate=1, n_rate=-1):
    for i in range(n_interactions):
        c.interact(p, n_rate)
        p.interact(c, p_rate)
    

In [7]:
c1 = l_consumers[100]
c2 = l_consumers[101]
c3 = l_consumers[102]
c4 = l_consumers[103]
c5 = l_consumers[104]
c6 = l_consumers[105]
c7 = l_consumers[106]
c8 = l_consumers[107]
c9 = l_consumers[108]
c10 = l_consumers[109]
c11 = l_consumers[110]

x = l_producers[200]
y = l_producers[202]
z = l_producers[204]
u = l_producers[206]
v = l_producers[208]

In [8]:
agree(c1, x, 15)

p_agree_c_disagree(c1, y, 1)
disagree(c1, y, 9)

p_agree_c_disagree(c1, z, 2)
disagree(c1, z, 8)

p_agree_c_disagree(c1, u, 1)
disagree(c1, u, 9)

p_agree_c_disagree(c1, v, 2)
disagree(c1, v, 8)

In [9]:
agree(c2, x, 15)

p_agree_c_disagree(c2, y, 1)
disagree(c2, y, 9)

p_agree_c_disagree(c2, z, 1)
disagree(c2, z, 9)

p_agree_c_disagree(c2, u, 2)
disagree(c2, u, 8)

p_agree_c_disagree(c2, v, 2)
disagree(c2, v, 8)


In [11]:
agree(c3, x, 15)

p_agree_c_disagree(c3, y, 2)
disagree(c3, y, 8)

p_agree_c_disagree(c3, z, 2)
disagree(c3, z, 8)

p_agree_c_disagree(c3, u, 1)
disagree(c3, u, 9)

p_agree_c_disagree(c3, v, 2)
disagree(c3, v, 8)

In [12]:
agree(c4, x, 15)
p_agree_c_disagree(c4, y, 1)
disagree(c4, y, 9)

p_agree_c_disagree(c4, z, 1)
disagree(c4, z, 9)

p_agree_c_disagree(c4, u, 1)
disagree(c4, u, 9)

p_agree_c_disagree(c4, v, 2)
disagree(c4, v, 8)

In [13]:
disagree(c5, x, 1)
p_agree_c_disagree(c5, y, 5)
agree(c5, y, 1)
agree(c5, z, 3)
p_agree_c_disagree(c5, z, 1)
disagree(c5, z, 1)

In [14]:
disagree(c6, x, 2)
agree(c6, y, 6)
agree(c6, z, 2)
p_agree_c_disagree(c6, z, 1)
disagree(c6, z, 1)

In [15]:
disagree(c7, x, 3)
agree(c7, y, 9)
disagree(c7, y, 4)
p_agree_c_disagree(c7, y, 1)
agree(c7, z, 5)

In [16]:
agree(c8, x, 15)
disagree(c8, u, 9)
p_agree_c_disagree(c8, u, 1)
disagree(c8, v, 9)
p_agree_c_disagree(c8, v, 1)

In [17]:
agree(c9, x, 15)
disagree(c9, u, 9)
p_agree_c_disagree(c9, u, 1)
disagree(c9, v, 9)
p_agree_c_disagree(c9, v, 1)

In [18]:
disagree(c10, x, 2)
agree(c10, u, 6)
disagree(c10, u, 3)
p_agree_c_disagree(c10, u, 1)
agree(c10, v, 6)
disagree(c10, v, 5)
p_agree_c_disagree(c10, u, 1)

In [19]:
agree(c11, y, 9)
agree(c11, z, 4)
disagree(c11, z, 3)
p_agree_c_disagree(c11, z, 1)

In [24]:

cum_sat = 0
cum_dis = 0
for _, cm in l_consumers.items():
#     print(cm.id)
    s, d = cm.my_bayes(x)
    print('For user = '+str(cm.label)+','+str(cm.id)+' S = '+str(s)+" D = "+str(d))
    
    cum_sat += s
    cum_dis += d
print(str(cum_sat)+'/'+str(cum_sat+ cum_dis))
bayes = (1 + cum_sat) / (2 + (cum_sat+ cum_dis))

For user = C1,100 S = 15.0 D = 0.0
For user = C2,101 S = 15.0 D = 0.0
For user = C3,102 S = 15.0 D = 0.0
For user = C4,103 S = 15.0 D = 0.0
For user = C5,104 S = 0.0 D = 1.0
For user = C6,105 S = 0.0 D = 2.0
For user = C7,106 S = 0.0 D = 3.0
For user = C8,107 S = 15.0 D = 0.0
For user = C9,108 S = 15.0 D = 0.0
For user = C10,109 S = 0.0 D = 2.0
For user = C11,110 S = 0 D = 0
90.0/98.0


In [25]:
bayes

0.91

In [26]:
c1.get_belief(x)

(0.9375, 0.0, 0.0625)

In [27]:
c2.get_belief(x)

(0.9375, 0.0, 0.0625)

In [28]:
c3.get_belief(x)

(0.9375, 0.0, 0.0625)

In [29]:
c4.get_belief(x)

(0.9375, 0.0, 0.0625)

In [30]:
c5.get_belief(x)

(0.0, 0.5, 0.5)

In [31]:
c6.get_belief(x)

(0.0, 0.6666666666666666, 0.3333333333333333)

In [32]:
c10.get_belief(x)

(0.0, 0.6666666666666666, 0.3333333333333333)

In [33]:
c9.get_belief(x)

(0.9375, 0.0, 0.0625)

In [34]:
c8.get_belief(x)

(0.9375, 0.0, 0.0625)

In [35]:
c11.get_belief(y)

(0.9, 0.0, 0.1)

In [36]:
c11.get_belief(z)

(0.4444444444444444, 0.4444444444444444, 0.1111111111111111)

In [37]:
c1.get_belief(y)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [38]:
c2.get_belief(y)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [39]:
c3.get_belief(y)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [40]:
c4.get_belief(y)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [41]:
c1.get_belief(z)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [42]:
c2.get_belief(z)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [43]:
c3.get_belief(z)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [44]:
c4.get_belief(z)

(0.0, 0.9090909090909091, 0.09090909090909091)

In [45]:
c5.get_belief(y)

(0.14285714285714285, 0.7142857142857143, 0.14285714285714285)

In [46]:
c5.get_belief(z)

(0.5, 0.3333333333333333, 0.16666666666666666)

In [47]:
c6.get_belief(y)

(0.8571428571428571, 0.0, 0.14285714285714285)

In [48]:
c6.get_belief(z)

(0.4, 0.4, 0.2)

In [49]:
c7.get_belief(y)

(0.6, 0.3333333333333333, 0.06666666666666667)

In [50]:
c7.get_belief(z)

(0.8333333333333334, 0.0, 0.16666666666666666)

In [51]:
c11.get_similarity(c1)

0.32026912680630204

In [52]:
c11.get_similarity(c2)

0.32026912680630204

In [53]:
c11.get_similarity(c3)

0.32026912680630204

In [54]:
c11.get_similarity(c4)

0.32026912680630204

In [55]:
c11.get_similarity(c5)

0.5835625323002815

In [56]:
c11.get_similarity(c6)

0.9400814106254535

In [57]:
c11.get_similarity(c7)

0.6312925098952105

In [58]:
# c8.get_belief(u)

In [59]:
u.get_belief(c8)

(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [60]:
# c9.get_belief(u)

In [61]:
u.get_belief(c9)

(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [62]:
u.get_belief(c10)

(0.6666666666666666, 0.25, 0.08333333333333333)

In [63]:
# c9.get_honesty()

In [64]:
v.get_belief(c8)

(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [65]:
v.get_belief(c9)

(0.09090909090909091, 0.8181818181818182, 0.09090909090909091)

In [66]:
v.get_belief(c10)

(0.5, 0.4166666666666667, 0.08333333333333333)

In [67]:
x.get_belief(c8)

(0.9375, 0.0, 0.0625)

In [68]:
x.get_belief(c9)

(0.9375, 0.0, 0.0625)

In [69]:
x.get_belief(c10)

(0.0, 0.6666666666666666, 0.3333333333333333)

In [70]:
#  c8.get_honesty()

In [71]:
c8.get_global_crediblity()

0.37310606060606055

In [72]:
c9.get_global_crediblity()

0.37310606060606055

In [73]:
c10.get_global_crediblity()

0.38888888888888884

In [74]:
all_v = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10]
all_p = [x,y,z,u,v]

In [75]:
# all_v
all_p

In [76]:
c11.p_g_reputation(x, all_v)

0.04663825757575756

In [77]:
def mod_BRS(provider, all_advisors, all_providers, i_S = 0.5, i_D = 0.5):
    rep = 0
    reputation = 0
    advisor_count = 0
    for a in all_advisors:
        if provider in a.b_satisfaction:
            advisor_count += 1
            e_p = (i_S + a.b_satisfaction[provider])/ (i_S + i_D + a.b_satisfaction[provider] + a.b_disatisfaction[provider])
            a_cred = 0
            sum_sat = 0
            sum_dis = 0
            for p in all_providers:
                if a in p.b_satisfaction:
                    sum_sat += p.b_satisfaction[a]
                    sum_dis += p.b_disatisfaction[a]

            if (sum_sat+sum_dis) > 0:
                a_cred = 1 - (sum_dis/(sum_sat+sum_dis))
            
            print('a '+str(a.label)+' for provider '+str(p.id)+' e_p :'+str(e_p))
            print('a '+str(a.label)+' a_cred :'+str(a_cred))
            
            rep += (e_p * a_cred)
    if advisor_count > 0:
        reputation = rep / advisor_count
    return reputation
    

In [78]:
mod_BRS(x, all_v, all_p)

a C1 for provider 208 e_p :0.96875
a C1 a_cred :0.38181818181818183
a C2 for provider 208 e_p :0.96875
a C2 a_cred :0.38181818181818183
a C3 for provider 208 e_p :0.96875
a C3 a_cred :0.4
a C4 for provider 208 e_p :0.96875
a C4 a_cred :0.36363636363636365
a C5 for provider 208 e_p :0.25
a C5 a_cred :0.8333333333333334
a C6 for provider 208 e_p :0.16666666666666666
a C6 a_cred :0.75
a C7 for provider 208 e_p :0.125
a C7 a_cred :0.6818181818181819
a C8 for provider 208 e_p :0.96875
a C8 a_cred :0.48571428571428577
a C9 for provider 208 e_p :0.96875
a C9 a_cred :0.48571428571428577
a C10 for provider 208 e_p :0.16666666666666666
a C10 a_cred :0.5833333333333333


0.2936399711399712

In [79]:
c11.p_g_reputation(x, all_v, alpha=0)

0.23319128787878785

In [88]:
c11.p_g_reputation(x, all_v, alpha=0.1, filter_threshold=0.3)

0.22702943374124668

In [89]:
c11.p_g_reputation(x, all_v, alpha=0.2, filter_threshold=0.3)

0.2208675796037055

In [90]:
c11.p_g_reputation(x, all_v, alpha=0.3, filter_threshold=0.3)

0.2147057254661643

In [91]:
c11.p_g_reputation(x, all_v, alpha=0.4, filter_threshold=0.3)

0.20854387132862315

In [92]:
c11.p_g_reputation(x, all_v, alpha=0.5, filter_threshold=0.3)

0.20238201719108195

In [93]:
c11.p_g_reputation(x, all_v, alpha=0.6, filter_threshold=0.3)

0.19622016305354079

In [94]:
c11.p_g_reputation(x, all_v, alpha=0.7, filter_threshold=0.3)

0.19005830891599962

In [95]:
c11.p_g_reputation(x, all_v, alpha=0.8, filter_threshold=0.3)

0.18389645477845845

In [96]:
c11.p_g_reputation(x, all_v, alpha=0.9, filter_threshold=0.3)

0.17773460064091726

In [97]:
c11.p_g_reputation(x, all_v, alpha=1, filter_threshold=0.3)

0.1715727465033761

In [1]:
import random

class Experiment:
    def __init__(self, scenario, nconsumer, nproducer, ntransactions, seed=100):
        self.scenario = scenario
        self.nconsumer = nconsumer
        self.nproducer = nproducer
        self.ntransactions = ntransactions
        self.nfaket = int(ntransactions/10)
        self.init_trans = ntransactions - self.nfaket
        self.consumers = {}
        self.producers = {}
        random.seed(seed)
        self.graphdata = {}
        self.graphdata_bayes = {}
        self.graphdata_modBRS = {}
        self.producer_id_start_range = 200
        self.consumer_id_start_range = 100
        self.alpha = 0
        self.start()
        
        
    def start(self, excludelist=[]):
        consumers = {}
        producers = {}
        for x in range(self.nconsumer):
            c = Consumer(self.consumer_id_start_range+x)
            consumers[c.id] = c
        for y in range(self.nproducer):
            p = Consumer(self.producer_id_start_range+y)
#             producers.append(p)
            producers[p.id] = p
        print('experiment'+str(self.scenario)+' initialised with'+str(len(consumers))+" c and "+str(len(producers))+" p")
        self.consumers = consumers
        self.producers = producers
        
        for _,c in consumers.items():
            p = random.choice(self.producers)
#             c_reps = []
#             c_reps_bayes = []
#             c_reps_modBRS = []
            
            c_dic = {}
            c_p_reps = []
            c_p_reps_bayes = []
            c_p_reps_modBRS = []
            
            if not p.id in excludelist and not c.id in excludelist:
                for i in range(self.init_trans):
                    r_i = randint(1,4)
                    if r_i == 1:
                        self.agree(c, p, 1)
                    elif r_i == 2:
                        self.disagree(c, p, 1)
                    elif r_i == 3:
                        self.c_agree_p_disagree(c, p, 1)
                    else:
                        self.p_agree_c_disagree(c, p, 1)
                    
                    if c.id in self.graphdata:
                        c_dic = self.graphdata[c.id]
                        if p.id in c_dic:
                            c_p_reps = c_dic[p.id]
                            
                    c_p_reps.append(c.p_g_reputation(p, self.consumers, self.alpha))
                    c_dic[p.id] = c_p_reps
                    self.graphdata[c.id] = c_dic
#                     p_c_reps.append(c.p_g_reputation(x, self.consumers, self.alpha))
    

    def agree(self, c, p, n_interactions, rate=1):
        for i in range(n_interactions):
            c.interact(p, rate)
            p.interact(c, rate)

    def disagree(self, c, p, n_interactions,rate=-1):
        agree(c, p, n_interactions,rate)

    def c_agree_p_disagree(self, c, p, n_interactions, p_rate=1, n_rate=-1):
        for i in range(n_interactions):
            c.interact(p, p_rate)
            p.interact(c, n_rate)

    def p_agree_c_disagree(self, c, p, n_interactions, p_rate=1, n_rate=-1):
        for i in range(n_interactions):
            c.interact(p, n_rate)
            p.interact(c, p_rate)
    
    def mod_BRS(self, provider, all_advisors, all_providers, i_S = 0.5, i_D = 0.5):
        rep = 0
        reputation = 0
        advisor_count = 0
        for a in all_advisors:
            if provider in a.b_satisfaction:
                advisor_count += 1
                e_p = (i_S + a.b_satisfaction[provider])/ (i_S + i_D + a.b_satisfaction[provider] + a.b_disatisfaction[provider])
                a_cred = 0
                sum_sat = 0
                sum_dis = 0
                for p in all_providers:
                    if a in p.b_satisfaction:
                        sum_sat += p.b_satisfaction[a]
                        sum_dis += p.b_disatisfaction[a]

                if (sum_sat+sum_dis) > 0:
                    a_cred = 1 - (sum_dis/(sum_sat+sum_dis))

                print('a '+str(a.label)+' for provider '+str(p.id)+' e_p :'+str(e_p))
                print('a '+str(a.label)+' a_cred :'+str(a_cred))

                rep += (e_p * a_cred)
        if advisor_count > 0:
            reputation = rep / advisor_count
        return reputation
    
    def bayes(self, l_consumers, xprovider):
        cum_sat = 0
        cum_dis = 0
        for _, cm in l_consumers.items():
        #     print(cm.id)
            s, d = cm.my_bayes(xprovider)
            print('For user = '+str(cm.label)+','+str(cm.id)+' S = '+str(s)+" D = "+str(d))

            cum_sat += s
            cum_dis += d
        print(str(cum_sat)+'/'+str(cum_sat+ cum_dis))
        bayes = (1 + cum_sat) / (2 + (cum_sat+ cum_dis))
        return bayes
    
    def ballot_stuffing(self, attacker_id, observers_id=[1], rating=0.9, n_fake_t=103, n_colluders=1, n_victims=5):
        colluders = []
        p = self.producers[attacker_id]
        few_producers =  []
        for i in range(n_colluders):
            colluder_id = self.consumer_id_start_range+i
            if not observers_id in observers_id:
                colluders.append(self.consumers[colluder_id])
            
        for c in colluders:
            self.agree(c, p, n_fake_t)
            p2 = random.choices(self.producers, k=n_victims)
            if p != p2:
                self.p_agree_c_disagree(c, p2, 1)
                self.disagree(c, p2, (randint(n_fake_t/10,n_fake_t/2)))
            
#             self.graphdata.append(t.reputation(p))
#             self.graphdata2.append(t.bayes(p))
            
    def bad_mouthing(self, victim, consumerid, rating=0.001, targetc=103, n_consumers=1):
        p = self.producers[victim]
        c = self.consumers[consumerid]
        c2 = self.consumers[consumerid+1]
        t = self.consumers[targetc]
        for x in range(self.nfaket):
            c.interact(p, rating)
            p.interact(c, random.uniform(0, 1))
            c2.interact(p, rating)
            p.interact(c2, random.uniform(0, 1))
            self.graphdata.append(t.reputation(p))
            self.graphdata2.append(t.bayes(p))
            
    def majority_descrimination(self, attacker, consumerid, rating=0.001, targetc=103, n_consumers=1):
        p = self.producers[attacker]
        t = self.consumers[targetc]
        for _, c in self.consumers.items():
            if c != t:
                for x in range(self.nfaket):
                    if c.id <= 104:
                        c.interact(p, 1)
                        p.interact(c, 1)
                    else:
                        c.interact(p, random.uniform(0, 0.5))
                        p.interact(c, 0)
                    self.graphdata.append(t.reputation(p))
                    self.graphdata2.append(t.bayes(p))
                                   
    def minority_descrimination(self, attacker, consumerid, rating=0.001, targetc=103, n_consumers=1):
        p = self.producers[attacker]
        t = self.consumers[targetc]
        for _, c in self.consumers.items():
            if c != t:
                for x in range(self.nfaket):
                    if c.id <= 104:
                        c.interact(p, 0)
                        p.interact(c, 0)
                    else:
                        c.interact(p, random.uniform(0.5, 1))
                        p.interact(c, random.uniform(0.5, 1))
                    self.graphdata.append(t.reputation(p))
                    self.graphdata2.append(t.bayes(p))
        
    
    def plot(self):
#         import matplotlib.pyplot as plt
#         xlables = [x for x in range(0, self.init_trans,10)]
#         fig = plt.figure()
#         ax = fig.add_subplot(111)
#         ax.plot([1, 2, 3, 4,5,6,7,8,9], xlables, color='lightblue', linewidth=3)
#         ax.scatter([0.3, 3.8, 1.2, 2.5, 2.5, 2.6, 2.7,3.0,3.5 ], [11, 25, 9,5,30,33,25, 26,35], color='darkgreen', marker='^')
#         ax.set_xlim(0.5, 4.5)
#         plt.show()
        import matplotlib.pyplot as plt
        import numpy as np
        
        print(self.graphdata)

        # Prepare the data
        x = np.linspace(0, len(self.graphdata), len(self.graphdata))

        # Plot the data
        plt.plot(x, self.graphdata2, alpha=0.7, label='Bayes')
        plt.plot(x, self.graphdata, alpha=0.7, label='ROBUSTRA')

        # Add a legend
        plt.legend()

        # Show the plot
        plt.show()
        